## Model for network of heat pump shown home

This code loads the required EPC/MCS data and generates a network for show homes with the given parameters. Gradio helps create an interactive interface for testing out different scenarios. 

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import gradio as gr

from show_homes import PROJECT_DIR, config
from show_homes.pipeline import show_homes_network
from show_homes.getters.show_home_data import get_show_home_data
from show_homes.config import config

from show_homes.utils import geo_utils

2023-01-25 16:18:29,974 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [3]:
df = get_show_home_data()

In [10]:
_ = show_homes_network.model_network(df, 'Any', True, 1, 0, 5, 6, 30, 'Orkney Islands', verbose=True)

Settings
************
Area: Orkney Islands
Property type: Any
Visitor ratio: 0.0
Host ratio: 0.01
Maximum visitors: 30
Maximum distance: 30
Same property type or not: True

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to /Users/juliasuter/Documents/repositories/show_homes/show_homes/analysis/maps/Generated_network_map_Any_True_0.01_0.0_30_30_Orkney_Islands.html!


In [4]:

def G(property_type, same_prop_type, 
                            host_ratio, visitor_ratio, 
                            v_max, n_open_days, d_max, local_auth, verbose=False):
    
    return show_homes_network.model_network(df, property_type, same_prop_type, 
                            host_ratio, visitor_ratio, 
                            v_max, n_open_days, d_max, local_auth, verbose=verbose)
 
    
property_types = ['Flat', 'Semi-detached House', 'Detached House', 'Terraced House', 'Any']
local_authorities = sorted(list(df['LOCAL_AUTHORITY_LABEL'].unique())) + ['GB']
    
demo = gr.Interface(
    fn=G,
    inputs=[gr.components.Radio(property_types, label='Property Type', value='Detached House'), 
            gr.components.Radio([True, False], label='Show home of same property', value=True), 
            gr.components.Slider(0, 100, value=1, step=1, label='Host ratio (%)'), 
            gr.components.Slider(0, 100, value=5, step=1, label='Visitor ratio (%)'),
            gr.components.Slider(1, 50, value=5, step=1, label='Max visitors'), 
            gr.components.Slider(1, 50, value=6, step=1, label='Number of slots/open days'), 
            gr.components.Slider(1, 50, value=35, step=1, label='Max distance'),
            gr.components.Dropdown(local_authorities, value='Orkney Islands', label='Local authorities')],
    outputs=["text", "html"],
    title='Network of Show Homes',)

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://cf46366f-28b0-4b7c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to /Users/juliasuter/Documents/repositories/show_homes/show_homes/analysis/maps/Generated_network_map_Detached_House_True_0.01_0.05_30_35_Orkney_Islands.html!
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to /Users/juliasuter/Documents/repositories/show_homes/show_homes/analysis/maps/Generated_network_map_Terraced_House_True_0.01_0.05_30_35_GB.html!
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to /Users/juliasuter/Documents/repositories/show_homes/show_homes/analysis/maps/Generated_network_map_Terraced_House_True_0.01_0.05_30_10_unknown.html!
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to /Users/juliasuter/Documents/repositories/show_homes/show_homes/analysis/maps/Generated_network_map_Terraced_House_True_0.01_0.0_30_10_unknown.html!
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to /Users/juliasuter/Documents/repositories/show_homes/s